In [25]:

#This dictionary has each of the system numbers as the keys with the query numbers again as keys and the rank again as a key
#With the doc number and score as a list of values
#Example - {1:{1:{1:[6567,5.0743]}}}
dictir = {}

#This dictionary has each of the query numbers as keys with the document numbers and relevance as key value pairs
#Example - {1:{9090 :3}}
retr = {}

#System numbers are ints, query number

#Reading all the files at once
for i in range(1, 7):
    filename = "systems/S" + str(i) + ".results"
    with open(filename) as fp:
        for line in fp:
            tokens = line.split(" ")
            #If we already have the system number in our dictionary
            if i in dictir:
                #If we already have the query number in our inner (2nd level) dictionary
                if int(tokens[0]) in dictir[i]:
                    #Upper the innermost (3rd level) dictionary
                    dictir[i][int(tokens[0])].update({int(tokens[3]): [int(tokens[2]), float(tokens[4])]})
                #Update the inner (2nd level) dictionary - adding the query number
                else:
                    dictir[i].update({int(tokens[0]): {int(tokens[3]): [int(tokens[2]), float(tokens[4])]}})
            #Update the dictionary - adding the system number
            else:
                dictir.update({i: {int(tokens[0]): {int(tokens[3]): [int(tokens[2]), float(tokens[4])]}}})
print(dictir)
#print(len(dictir[2][5]))

{1: {1: {1: [6567, 5.0743], 2: [9652, 4.4829], 3: [9684, 4.3478], 4: [7844, 4.3268], 5: [9584, 4.216], 6: [7853, 4.0384], 7: [9090, 4.0021], 8: [9574, 4.0011], 9: [6850, 3.9796], 10: [9616, 3.9646], 11: [9281, 3.8694], 12: [9891, 3.8448], 13: [9665, 3.8105], 14: [8446, 3.8105], 15: [9591, 3.7642], 16: [9198, 3.7352], 17: [5690, 3.6808], 18: [5493, 3.6772], 19: [6915, 3.6539], 20: [6171, 3.6375], 21: [5023, 3.6375], 22: [7732, 3.6234], 23: [8871, 3.5779], 24: [8124, 3.5058], 25: [6602, 3.485], 26: [7353, 3.447], 27: [8564, 3.3441], 28: [8419, 3.3176], 29: [7735, 3.2894], 30: [7730, 3.2745], 31: [9069, 3.2184], 32: [8179, 3.2013], 33: [7745, 3.2], 34: [9625, 3.1213], 35: [9394, 3.1213], 36: [9156, 3.1213], 37: [9961, 3.1156], 38: [5936, 3.1156], 39: [5014, 3.0769], 40: [7811, 3.0696], 41: [6675, 3.0696], 42: [7980, 3.0168], 43: [7553, 2.9637], 44: [6473, 2.9541], 45: [8270, 2.9316], 46: [7771, 2.9316], 47: [8375, 2.8927], 48: [5660, 2.8927], 49: [9530, 2.8756], 50: [5368, 2.8756], 51: [7

In [47]:
import operator
sorted_x = {}

with open("systems/qrels.txt") as fr:
    for line in fr:
        #Split the line on spaces first
        tokens = line.split()
        #Get the query number first
        qno = int(tokens[0].split(":")[0])

        #Take out the query number from the list now
        tokens.remove(tokens[0])
        for token in tokens:
            #Get the document numbers and relevance value
            qtok = token.split(",")
            docno = int(qtok[0].replace('(', ""))
            rele = int(qtok[1].rstrip().replace(')', ""))

            #If we already have the query number in our dictionary
            if qno in retr:
                retr[qno].update({docno: rele})
            else:
                retr.update({qno: {docno: rele}})
for x in retr:
    sorted_x[x] = sorted(retr[x].items(), key=operator.itemgetter(1), reverse = True)

print(sorted_x)

{1: [(9090, 3), (9574, 2), (6850, 2), (5011, 1), (9684, 1), (8709, 1)], 2: [(5715, 2), (5766, 2), (9677, 2), (6498, 1), (5653, 1), (6327, 1), (7117, 1), (6079, 1)], 3: [(9743, 3)], 4: [(5269, 3), (6491, 3), (8032, 3), (9444, 3), (7435, 2), (8988, 2), (9445, 2), (5883, 2), (7620, 1), (10117, 1), (4742, 1), (10119, 1), (6743, 1), (9739, 1), (9038, 1), (6334, 1), (6288, 1), (6289, 1), (9746, 1), (6675, 1), (5268, 1), (7046, 1), (5783, 1), (9523, 1), (8414, 1), (9744, 1), (8315, 1), (6054, 1), (9745, 1), (5865, 1), (10029, 1), (6382, 1), (8562, 1), (7827, 1), (6836, 1), (9278, 1), (7224, 1), (9720, 1), (6292, 1), (8318, 1)], 5: [(1646, 1), (8646, 1), (4983, 1), (8282, 1), (3111, 1), (6669, 1), (2126, 1)], 6: [(6736, 3), (8433, 3), (7487, 3), (7424, 1), (9541, 1), (8593, 1), (9736, 1), (8844, 1), (7737, 1), (8120, 1), (5305, 1), (8261, 1)], 7: [(7646, 3), (3156, 2), (4144, 1)], 8: [(9891, 3), (7844, 3), (9684, 2), (9574, 2), (6171, 1), (9281, 1), (9090, 1), (9652, 1)], 9: [(5121, 3), (6923,

In [65]:
import math
import operator

sorted_retr = {}

#We have to sort out retr dictionary of related documents for queries to get the ideal gain at every step
for x in retr:
    sorted_retr[x] = sorted(retr[x].items(), key=operator.itemgetter(1), reverse = True)
print(sorted_retr)


#Each system
for i in dictir:
    #Each query
    for j in dictir[i]:
        precsum = 0
        #Precision at 10
        for k in range(1, 11):
            #If the document number is in the list of relevant documents for the query
            if dictir[i][j][k][0] in retr[j]:
                precsum = precsum + 1
        precsum = precsum/10.00
        #print(i, j)
        #print(precsum)
        #print("\n")
        
        #Recall at 50
        recsum = 0
        for k in range(1, 51):
            #If the document number is in the list of relevant documents for the query
            if dictir[i][j][k][0] in retr[j]:
                recsum = recsum + 1
        #What fraction of the relevant documents are retrieved?
        recsum = float(recsum)/len(retr[j])
        #print(i, j)
        #print(recsum)
        #print("\n")
        
        #nDCG@10
        #For getting the ID gain
        counter = 0
        dcg = 0
        idcg = 0
        for k in range(1, 11):
            
            #i,j,k are system, query and rank
            docID = dictir[i][j][k][0]
            if docID in retr[j]:
                #Gain is the releavnce of that specific document
                gain = retr[j][docID]
            else:
                #Else relevance is 0
                gain = 0.0
                
            #The first gain value in  the sorted list
            #Only if the we have not covered all the relevant documents
            if (counter >= len(sorted_retr[j])):
                id_gain = 0.0
            else:
                id_gain = sorted_retr[j][counter][1]
                
            #print(gain)
            #print(id_gain)
            counter = counter + 1
            if k == 1:
                dcg = dcg + gain
                idcg = idcg + id_gain
            else:
                #We do the log of the rank too
                dcg = dcg + (float(gain)/math.log2(k))
                idcg = idcg + (float(id_gain)/math.log2(k))
        ndcg = round(dcg/idcg, 2)
        print(ndcg)

{1: [(9090, 3), (9574, 2), (6850, 2), (5011, 1), (9684, 1), (8709, 1)], 2: [(5715, 2), (5766, 2), (9677, 2), (6498, 1), (5653, 1), (6327, 1), (7117, 1), (6079, 1)], 3: [(9743, 3)], 4: [(5269, 3), (6491, 3), (8032, 3), (9444, 3), (7435, 2), (8988, 2), (9445, 2), (5883, 2), (7620, 1), (10117, 1), (4742, 1), (10119, 1), (6743, 1), (9739, 1), (9038, 1), (6334, 1), (6288, 1), (6289, 1), (9746, 1), (6675, 1), (5268, 1), (7046, 1), (5783, 1), (9523, 1), (8414, 1), (9744, 1), (8315, 1), (6054, 1), (9745, 1), (5865, 1), (10029, 1), (6382, 1), (8562, 1), (7827, 1), (6836, 1), (9278, 1), (7224, 1), (9720, 1), (6292, 1), (8318, 1)], 5: [(1646, 1), (8646, 1), (4983, 1), (8282, 1), (3111, 1), (6669, 1), (2126, 1)], 6: [(6736, 3), (8433, 3), (7487, 3), (7424, 1), (9541, 1), (8593, 1), (9736, 1), (8844, 1), (7737, 1), (8120, 1), (5305, 1), (8261, 1)], 7: [(7646, 3), (3156, 2), (4144, 1)], 8: [(9891, 3), (7844, 3), (9684, 2), (9574, 2), (6171, 1), (9281, 1), (9090, 1), (9652, 1)], 9: [(5121, 3), (6923,

In [54]:
import math
import operator

sorted_retr = {}

#We have to sort out retr dictionary of related documents for queries to get the ideal gain at every step
for x in retr:
    sorted_retr[x] = sorted(retr[x].items(), key=operator.itemgetter(1), reverse = True)
print(sorted_retr)

#nDCG@10
#For getting the ID gain
counter = 0
dcg = 0
idcg = 0
for k in range(1, 11):
    #i,j,k are system, query and rank
    docID = dictir[i][j][k][0]
    if docID in sorted_retr[j]:
        #Gain is the releavnce of that specific document
        gain = sorted_retr[j][docID]
    else:
        #Else relevance is 0
        gain = 0.0
    #The first gain value in  the sorted list
    id_gain = sorted_retr[i][counter][1]
    counter = counter + 1
    if k == 1:
        dcg = dcg + gain
        idcg = idcg + id_gain
    else:
        #We do the log of the rank too
        dcg = dcg + (float(gain)/math.log2(k))
        idcg = idcg + (float(id_gain)/math.log2(k))
    ndcg = round(dcg/idcg, 2)
    print(ndcg)
    #fq.write(str(ndcg))
    #fq.write("    ")
    #fq.write("\n")


{1: [(9090, 3), (9574, 2), (6850, 2), (5011, 1), (9684, 1), (8709, 1)], 2: [(5715, 2), (5766, 2), (9677, 2), (6498, 1), (5653, 1), (6327, 1), (7117, 1), (6079, 1)], 3: [(9743, 3)], 4: [(5269, 3), (6491, 3), (8032, 3), (9444, 3), (7435, 2), (8988, 2), (9445, 2), (5883, 2), (7620, 1), (10117, 1), (4742, 1), (10119, 1), (6743, 1), (9739, 1), (9038, 1), (6334, 1), (6288, 1), (6289, 1), (9746, 1), (6675, 1), (5268, 1), (7046, 1), (5783, 1), (9523, 1), (8414, 1), (9744, 1), (8315, 1), (6054, 1), (9745, 1), (5865, 1), (10029, 1), (6382, 1), (8562, 1), (7827, 1), (6836, 1), (9278, 1), (7224, 1), (9720, 1), (6292, 1), (8318, 1)], 5: [(1646, 1), (8646, 1), (4983, 1), (8282, 1), (3111, 1), (6669, 1), (2126, 1)], 6: [(6736, 3), (8433, 3), (7487, 3), (7424, 1), (9541, 1), (8593, 1), (9736, 1), (8844, 1), (7737, 1), (8120, 1), (5305, 1), (8261, 1)], 7: [(7646, 3), (3156, 2), (4144, 1)], 8: [(9891, 3), (7844, 3), (9684, 2), (9574, 2), (6171, 1), (9281, 1), (9090, 1), (9652, 1)], 9: [(5121, 3), (6923,